In [1]:
# import necessary library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics
import pickle

In [2]:
# load data
data = pd.read_csv("Housing.csv")
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


# data preprocessing
- duplicate
- null
- data type
- outlier
- data scaling
- data encoding

In [3]:
# create a data back up
df= data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [4]:
# duplicate
df[df.duplicated()]

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus


In [5]:
# Null Values
df.isnull().sum()

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

In [6]:
df['furnishingstatus'].unique()

array(['furnished', 'semi-furnished', 'unfurnished'], dtype=object)

In [7]:
# non numeric columns
non_numeric_columns = df.select_dtypes(include=object).columns.to_list()
non_numeric_columns

['mainroad',
 'guestroom',
 'basement',
 'hotwaterheating',
 'airconditioning',
 'prefarea',
 'furnishingstatus']

In [8]:
# Outlier
df.describe()   # no outliers here

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [9]:
# data encoding
label = LabelEncoder()
for i in non_numeric_columns:
    df[i] = label.fit_transform(df[i])
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,0


In [10]:
# Setting feature and target
X = df.drop(columns= 'price', axis=1)
Y = df['price']

# split dataset
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size= 0.2, random_state=42)

# scale data
scaler = StandardScaler()
X_Train_scaled = scaler.fit_transform(X_Train)
X_Test_scaled = scaler.fit_transform(X_Test)

## Linear Regression model


In [11]:
# build and train model
model = LinearRegression()
model.fit(X_Train_scaled, Y_Train)

# Prediction
Y_Predict = model.predict(X_Test_scaled)

# model performance
mae = metrics.mean_absolute_error(Y_Test, Y_Predict)
rmse = metrics.root_mean_squared_error(Y_Test, Y_Predict)
r2 = metrics.r2_score(Y_Test, Y_Predict)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Root Mean Absolute Error: {rmse:.2f}')
print(f'R2 score: {r2:.2f}')

Mean Absolute Error: 970968.88
Root Mean Absolute Error: 1350749.47
R2 score: 0.64


## Ridge model

In [12]:
# build and train model
ridge = Ridge(alpha=0.1)
ridge.fit(X_Train_scaled, Y_Train)

# prediction
Y_Predict = ridge.predict(X_Test_scaled)

# model performance
mae = metrics.mean_absolute_error(Y_Test, Y_Predict)
rmse = metrics.root_mean_squared_error(Y_Test, Y_Predict)
r2 = metrics.r2_score(Y_Test, Y_Predict)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Root Mean Absolute Error: {rmse:.2f}')
print(f'R2 score: {r2:.2f}')

Mean Absolute Error: 970971.05
Root Mean Absolute Error: 1350780.30
R2 score: 0.64


## Lasso model

In [13]:
# build and train model
lasso = Lasso(alpha=1.0)
lasso.fit(X_Train_scaled, Y_Train)

# prediction
Y_Predict = lasso.predict(X_Test_scaled)

# model performance
mae = metrics.mean_absolute_error(Y_Test, Y_Predict)
rmse = metrics.root_mean_squared_error(Y_Test, Y_Predict)
r2 = metrics.r2_score(Y_Test, Y_Predict)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Root Mean Absolute Error: {rmse:.2f}')
print(f'R2 score: {r2:.2f}')

Mean Absolute Error: 970969.28
Root Mean Absolute Error: 1350750.24
R2 score: 0.64


## Decision Tree Model

In [14]:
# build and train model
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_Train_scaled, Y_Train)

# prediction
Y_Predict = dt.predict(X_Test_scaled)

# model performance
mae = metrics.mean_absolute_error(Y_Test, Y_Predict)
rmse = metrics.root_mean_squared_error(Y_Test, Y_Predict)
r2 = metrics.r2_score(Y_Test, Y_Predict)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Root Mean Absolute Error: {rmse:.2f}')
print(f'R2 score: {r2:.2f}')

Mean Absolute Error: 1322614.68
Root Mean Absolute Error: 1760487.87
R2 score: 0.39


## Random Forest Model

In [15]:
# build and train model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_Train_scaled, Y_Train)

# prediction
Y_Predict = rf.predict(X_Test_scaled)

# model performance
mae = metrics.mean_absolute_error(Y_Test, Y_Predict)
rmse = metrics.root_mean_squared_error(Y_Test, Y_Predict)
r2 = metrics.r2_score(Y_Test, Y_Predict)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Root Mean Absolute Error: {rmse:.2f}')
print(f'R2 score: {r2:.2f}')

Mean Absolute Error: 1043853.59
Root Mean Absolute Error: 1414055.52
R2 score: 0.60


# Gradient Boost Model

In [16]:
# build and train model
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05, random_state=42)
gbr.fit(X_Train_scaled, Y_Train)

# prediction
Y_Predict = rf.predict(X_Test_scaled)

# model performance
mae = metrics.mean_absolute_error(Y_Test, Y_Predict)
rmse = metrics.root_mean_squared_error(Y_Test, Y_Predict)
r2 = metrics.r2_score(Y_Test, Y_Predict)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Root Mean Absolute Error: {rmse:.2f}')
print(f'R2 score: {r2:.2f}')

Mean Absolute Error: 1043853.59
Root Mean Absolute Error: 1414055.52
R2 score: 0.60


# Deploying Linear Regression model

In [17]:
with open('housing.pkl', 'wb') as file:
    pickle.dump(model, file)